<div class='alert alert-info' style='padding:0'>
<h2 style='margin:20px; text-align:center'>Validación Cruzada</h2>
</div>

In [1]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, KFold

In [2]:
data = pd.read_csv('./datasets/felicidad.csv')
data.head()

,country,rank,score,high,low,gdp,family,lifexp,freedom,generosity,corruption,dystopia
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [3]:
x = data.drop(['country', 'score'], axis=1)
y = data['score']

In [4]:
model = DecisionTreeRegressor()
score = cross_val_score(model, x, y, scoring='neg_mean_squared_error', cv=5)
score

array([-0.4217934 , -0.05220192, -0.04694546, -0.07061734, -0.41386293])

Entre mas cercano a cero mejor es el modelo

In [5]:
np.abs(np.mean(score)) # score promedio

0.20108421024254702

In [6]:
kf = KFold(n_splits=3, shuffle=True, random_state=42)
for train, test in kf.split(data):
    print(f'\nTrain: {train}\n\n Test: {test}\n')
    print('='*32)


Train: [  0   1   2   3   4   5   6   7   8  10  13  14  16  17  20  21  23  25
  28  32  33  34  35  37  38  39  40  41  43  44  46  47  48  49  50  52
  53  54  57  58  59  61  62  63  64  67  70  71  72  73  74  77  80  83
  87  88  89  91  92  94  97  98  99 100 101 102 103 104 105 106 107 108
 110 111 112 113 114 115 116 120 121 123 125 127 128 129 130 132 134 135
 136 139 140 143 144 145 146 148 149 150 151 152 154]

 Test: [  9  11  12  15  18  19  22  24  26  27  29  30  31  36  42  45  51  55
  56  60  65  66  68  69  75  76  78  79  81  82  84  85  86  90  93  95
  96 109 117 118 119 122 124 126 131 133 137 138 141 142 147 153]


Train: [  1   2   3   6   8   9  11  12  13  14  15  17  18  19  20  21  22  24
  26  27  29  30  31  36  37  38  42  45  48  50  51  52  54  55  56  57
  58  59  60  63  65  66  68  69  71  72  74  75  76  78  79  81  82  83
  84  85  86  87  88  89  90  91  92  93  95  96  99 100 102 103 106 107
 109 112 115 116 117 118 119 120 121 122 124 126 128

Cada array tiene los inices de los datos en el dataset segun cada partición

# Optimizacion Paramétrica

<ul>
<li>Optimizacion manual</li>
    <ol>
        <li>Escojer el modelo para ajustar.</li>
        <li>Buscar en la documentación de sklearn</li>
        <li>Identificar los posibles ajustes.</li>
        <li>Probar combinaciones una por un aiterando a través de listas</li>
    </ol><br />
<li>Grilla de parametros (GridSeachCV)</li>
    <ol>
        <li>Definir una o varias variables que queremos optimizar</li>
        <li>Identificar los posibles valores que pueden tener los parametros</li>
        <li>Crear un diccionario de parametros</li>
        <li>Usar cross validation</li>
        <li>Entrenar el modelo.</li>
    </ol><br />
<li>Búsqueda aleatoria (RandomizedSeacrhCV)</li>
    <ol>
        <li>Definir una o varias variables que queremos optimizar</li>
        <li>Identificar los rangos de valores que pueden tomar ciertos parametros</li>
        <li>Crear un diccionario de rangos de valores</li>
        <li>Usar cross validation</li>
        <li>Entrenar el modelo.</li>
    </ol><br />
</ul>

In [11]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

### RandomizedSeacrhCV

In [15]:
x = data.drop(['country', 'score', 'rank'], axis=1)
y = data['score']

reg = RandomForestRegressor()
params = {
    'n_estimators': range(4, 16), # Escoge el mejor numero de estimadores
    'criterion': ['mse', 'mae'],  # Medida de calidad de los splits
    'max_depth': range(2, 11)     # Profundidad del arbol
}

rand_est = RandomizedSearchCV(
        reg,
        params,
        n_iter=10,
        cv=3,
        scoring='neg_mean_absolute_error',
        iid=True
)

rand_est.fit(x, y)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=None,
          param_distributions={'n_estimators': range(4, 16), 'criterion': ['mse', 'mae'], 'max_depth': range(2, 11)},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='neg_mean_absolute_error',
          verbose=0)

In [18]:
print(f'Best Estimator:\n {rand_est.best_estimator_}')

Best Estimator:
 RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=6,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=4, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)


In [17]:
print(f'Best Params:\n {rand_est.best_params_}')

Best Params:
 {'n_estimators': 4, 'max_depth': 6, 'criterion': 'mae'}


In [20]:
print(f'Prediction:\n {rand_est.predict(x.loc[[0]])}')

Prediction:
 [7.52625012]
